In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
%matplotlib inline

### 베이즈 정리
- P(H|E) = P(E|H) x P(H) / P(E)

In [3]:
apparel = ['y','n','y','y','y','y','y','n','y','y','y','y']
leisure = ['n','y','n','n','y','y','n','y','y','n','n','y']
furniture = ['n','n','n','y','y','y','y','n','n','n','n','y']
kitchenware = ['n','n','y','y','y','y','y','y','n','y','n','n']
gender = ['m','m','f','f','m','f','f','f','m','f','m','f']

In [8]:
buy = pd.DataFrame([apparel, leisure, furniture, kitchenware, gender], index=['apparel', 'leisure', 'furniture', 'kitchenware', 'gender']).T
buy

,apparel,leisure,furniture,kitchenware,gender
0,y,n,n,n,m
1,n,y,n,n,m
2,y,n,n,y,f
3,y,n,y,y,f
4,y,y,y,y,m
5,y,y,y,y,f
6,y,n,y,y,f
7,n,y,n,y,f
8,y,y,n,n,m
9,y,n,n,y,f


In [14]:
cust_Q = pd.DataFrame([['y', 'y', 'n', 'y', None]], columns=['apparel', 'leisure', 'furniture', 'kitchenware', 'gender'])
cust_Q

,apparel,leisure,furniture,kitchenware,gender
0,y,y,n,y,None


### 고객 Q가 상품을 구매한 사건 $E_q : (A=y) and (L=y) and (F=n) and (K=y)$
* A:  apparel, L:  leisure, F:  furniture, K:  kitchenware

### 고객 Q가 남자일 확률 $P(G=m | E_q)$
$P(G=m | E_Q) = \frac{P(E_Q|G=m)\times P(G=m)}{P(E_Q)}$

In [146]:
temp = buy.melt(id_vars='gender', var_name='category')
temp['c'] = 1
temp = temp.pivot_table(index='value', columns=['category','gender'], aggfunc='sum').sort_index(ascending=False)
temp

c                                          
category apparel    furniture    kitchenware    leisure   
gender         f  m         f  m           f  m       f  m
value                                                     
y              6  4         4  1           6  1       3  3
n              1  1         3  4           1  4       4  2

In [147]:
temp.columns =  pd.MultiIndex.from_tuples([col[1:] for col in temp.columns])
temp

apparel    furniture    kitchenware    leisure   
            f  m         f  m           f  m       f  m
value                                                  
y           6  4         4  1           6  1       3  3
n           1  1         3  4           1  4       4  2

In [148]:
temp = temp.swaplevel(axis=1).sort_index(level=[1, 0], axis=1, ascending=[True, False]).iloc[:,[0,1,6,7,2,3,4,5]]
temp

,m,f,m,f,m,f,m,f
,apparel,apparel,leisure,leisure,furniture,furniture,kitchenware,kitchenware
value,,,,,,,,
y,4,6,3,3,1,4,1,6
n,1,1,2,4,4,3,4,1


In [156]:
temp = pd.concat([temp, pd.DataFrame([[f'{temp.iloc[i, j]}/{temp.sum()[j]}' for j in range(temp.shape[1])] for i in range(2)], columns=temp.columns, index=['y_rate', 'n_rate'])])
temp

,m,f,m,f,m,f,m,f
,apparel,apparel,leisure,leisure,furniture,furniture,kitchenware,kitchenware
y,4,6,3,3,1,4,1,6
n,1,1,2,4,4,3,4,1
y_rate,4/5,6/7,3/5,3/7,1/5,4/7,1/5,6/7
n_rate,1/5,1/7,2/5,4/7,4/5,3/7,4/5,1/7


$P(E_Q|G=m)=P(A=y|G=m)\times P(L=y|G=m)\times P(F=n|G=m)\times P(K=y|G=m)$

$P(E_Q|G=m)=\frac{4}{5}\times\frac{3}{5}\times\frac{4}{5}\times\frac{1}{5} = \frac{48}{625} = 0.0768$

$P(G=m|E_Q) = \frac{0.0768 \times 0.4167}{P(E_Q)} = \frac{0.0320}{P(E_Q)}$

### 고객 Q가 여자일 확률 $P(G=f | E_Q)$

$P(G=f|E_Q) = \frac{P(E_Q|G=f) \times P(G=f)}{P(E_Q)}$

$P(E_Q|G=f)=P(A=y|G=f)\times P(L=y|G=f)\times P(F=n|G=f)\times P(K=y|G=f) = \frac{6}{7} \times \frac{3}{7}\times \frac{3}{7}\times \frac{6}{7} = \frac{324}{2401} = 0.1349$

$P(G=f|E_Q) = \frac{0.1349 \times 0.5733}{P(E_Q)} = \frac{0.0787}{P(E_Q)}$

### $P(E_Q)$

$P(E_Q) = P(E_Q|G=m)\times P(G=m) + P(E_Q|G=f)\times P(G=f)$

$P(E_Q) = 0.0320+0.0787 = 0.1107$

$P(G=m|E_Q) = \frac{0.0320}{0.1107} = 0.2891$

$P(G=f|E_Q) = \frac{0.0787}{0.1107} = 0.7109$

$P(G=f|E_Q) > P(G=m|E_Q)$

In [166]:
buy['age'] = [40,39,35,30,45,31,38,29,28,34,46,42]
buy

,apparel,leisure,furniture,kitchenware,gender,age
0,y,n,n,n,m,40
1,n,y,n,n,m,39
2,y,n,n,y,f,35
3,y,n,y,y,f,30
4,y,y,y,y,m,45
5,y,y,y,y,f,31
6,y,n,y,y,f,38
7,n,y,n,y,f,29
8,y,y,n,n,m,28
9,y,n,n,y,f,34


### 고객 T가 상품을 구매한 사건 $E_T : (Age=44) and (A=y)and(L=y)and(F=n)and(K=y)$

In [167]:
buy.groupby('gender').agg({'age':['mean', 'std']})

age          
             mean       std
gender                     
f       34.142857  4.670067
m       39.600000  7.162402

### 고객 T가 남성인 경우

In [174]:
import scipy as sp
rv = sp.stats.norm(39.60, 7.16)
rv.pdf(44)

0.04613104686742316

In [182]:
rv.cdf(44) - rv.cdf(43)

0.04800907948339117

$P(G=m|E_T) = \frac{0.0768 \times 0.4167}{P(E_T)} \times 0.0461 = \frac{0.0015}{P(E_T)}$

### 고객 T가 여성인 경우

In [183]:
rv = sp.stats.norm(34.14, 4.67)
rv.pdf(44)

0.00919592358557332

$P(G=m|E_T) = \frac{0.1349 \times 0.5833}{P(E_T)} \times 0.0092 = \frac{0.0007}{P(E_T)}$

$P(G=m|E_T) > P(G=f|E_T)$

# 베이즈 분류기를 이용한 스팸 메일 판정

In [211]:
with open('spambase.names', 'r') as file:
    colnames = ['word_' + txt.split(': ')[0].split('freq_')[1] for txt in file.readlines()[33:-3]]
colnames.extend(['acap', 'lcap', 'tcap', 'spam'])
colnames

['word_make',
 'word_address',
 'word_all',
 'word_3d',
 'word_our',
 'word_over',
 'word_remove',
 'word_internet',
 'word_order',
 'word_mail',
 'word_receive',
 'word_will',
 'word_people',
 'word_report',
 'word_addresses',
 'word_free',
 'word_business',
 'word_email',
 'word_you',
 'word_credit',
 'word_your',
 'word_font',
 'word_000',
 'word_money',
 'word_hp',
 'word_hpl',
 'word_george',
 'word_650',
 'word_lab',
 'word_labs',
 'word_telnet',
 'word_857',
 'word_data',
 'word_415',
 'word_85',
 'word_technology',
 'word_1999',
 'word_parts',
 'word_pm',
 'word_direct',
 'word_cs',
 'word_meeting',
 'word_original',
 'word_project',
 'word_re',
 'word_edu',
 'word_table',
 'word_conference',
 'word_;',
 'word_(',
 'word_[',
 'word_!',
 'word_$',
 'word_#',
 'acap',
 'lcap',
 'tcap',
 'spam']

In [208]:
raw = pd.read_csv('spambase.data', header=None)
raw

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.000,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.000,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.010,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.000,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.000,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4596,0.31,0.00,0.62,0.0,0.00,0.31,0.00,0.00,0.00,0.00,...,0.000,0.232,0.0,0.000,0.000,0.000,1.142,3,88,0
4597,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.000,0.000,0.0,0.353,0.000,0.000,1.555,4,14,0
4598,0.30,0.00,0.30,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.102,0.718,0.0,0.000,0.000,0.000,1.404,6,118,0
4599,0.96,0.00,0.00,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.000,0.057,0.0,0.000,0.000,0.000,1.147,5,78,0


In [212]:
raw.columns = colnames
raw

,word_make,word_address,word_all,word_3d,word_our,word_over,word_remove,word_internet,word_order,word_mail,...,word_;,word_(,word_[,word_!,word_$,word_#,acap,lcap,tcap,spam
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.000,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.000,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.010,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.000,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.000,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4596,0.31,0.00,0.62,0.0,0.00,0.31,0.00,0.00,0.00,0.00,...,0.000,0.232,0.0,0.000,0.000,0.000,1.142,3,88,0
4597,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.000,0.000,0.0,0.353,0.000,0.000,1.555,4,14,0
4598,0.30,0.00,0.30,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.102,0.718,0.0,0.000,0.000,0.000,1.404,6,118,0
4599,0.96,0.00,0.00,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.000,0.057,0.0,0.000,0.000,0.000,1.147,5,78,0


In [238]:
from sklearn.model_selection import train_test_split

In [248]:
i = 0
X = raw.drop('spam', axis=1)
y = raw.spam
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.6001, random_state=i)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, train_size=0.5, random_state=i)

### LDA 모델

In [253]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.metrics import confusion_matrix

In [254]:
lda_model = LDA().fit(X_train, y_train)

In [259]:
lda_model.classes_

array([0, 1], dtype=int64)

In [256]:
cm_lda = confusion_matrix(y_train, lda_model.predict(X_train))
cm_lda

array([[1614,   78],
       [ 234,  835]], dtype=int64)

In [266]:
round((cm_lda[0][0] + cm_lda[1][1]) / cm_lda.sum()*100, 2)

88.7

### Naive Bayes 모델

In [283]:
from sklearn.naive_bayes import *

In [261]:
nb_model = BernoulliNB().fit(X_train, y_train)

In [262]:
nb_model.classes_

array([0, 1], dtype=int64)

In [263]:
cm_nb = confusion_matrix(y_train, nb_model.predict(X_train))
cm_nb

array([[1582,  110],
       [ 199,  870]], dtype=int64)

In [265]:
round((cm_nb[0][0] + cm_nb[1][1]) / cm_nb.sum()*100, 2)

88.81

In [267]:
def get_acc(x,y,model):
    cm = confusion_matrix(y, model.predict(x))
    acc = round((cm[0][0] + cm[1][1])/cm.sum()*100,2)
    return acc

In [334]:
# Bernoulli
result = pd.DataFrame()
for i in range(10):
    X = raw.drop('spam', axis=1)
    y = raw.spam
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.6001, random_state=i)
    X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, train_size=0.5, random_state=i)
    
    lda_model = LDA().fit(X_train, y_train)
    lda_train_acc = get_acc(X_train, y_train, lda_model)
    lda_val_acc = get_acc(X_val, y_val, lda_model)
    
    nb_model = BernoulliNB().fit(X_train, y_train)
    nb_train_acc = get_acc(X_train, y_train, nb_model)
    nb_val_acc = get_acc(X_val, y_val, nb_model)
    result = result.append({'nb_train':nb_train_acc, 'lda_train':lda_train_acc, 'nb_val':nb_val_acc, 'lda_val':lda_val_acc}, ignore_index=True)
result_avg = result.mean()
result_avg.name = '평균'
result = result.append(result_avg)
result

,nb_train,lda_train,nb_val,lda_val
0,88.810,88.700,88.480,88.480
1,88.770,88.590,88.910,88.910
2,88.700,88.920,88.800,87.500
3,88.340,88.700,89.240,87.500
4,88.340,89.970,89.570,89.240
5,88.660,89.280,90.110,89.570
6,89.530,88.920,87.390,88.260
7,89.210,89.500,88.700,90.650
8,88.920,88.700,88.480,89.350
9,88.840,88.770,90.110,90.110


In [335]:
# Gaussian
result = pd.DataFrame()
for i in range(10):
    X = raw.drop('spam', axis=1)
    y = raw.spam
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.6001, random_state=i)
    X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, train_size=0.5, random_state=i)
    
    lda_model = LDA().fit(X_train, y_train)
    lda_train_acc = get_acc(X_train, y_train, lda_model)
    lda_val_acc = get_acc(X_val, y_val, lda_model)
    
    nb_model = GaussianNB().fit(X_train, y_train)
    nb_train_acc = get_acc(X_train, y_train, nb_model)
    nb_val_acc = get_acc(X_val, y_val, nb_model)
    result = result.append({'nb_train':nb_train_acc, 'lda_train':lda_train_acc, 'nb_val':nb_val_acc, 'lda_val':lda_val_acc}, ignore_index=True)
result_avg = result.mean()
result_avg.name = '평균'
result = result.append(result_avg)
result

,nb_train,lda_train,nb_val,lda_val
0,83.160,88.700,83.480,88.480
1,82.040,88.590,83.480,88.910
2,81.960,88.920,82.390,87.500
3,82.870,88.700,81.740,87.500
4,82.650,89.970,82.610,89.240
5,81.820,89.280,80.650,89.570
6,81.750,88.920,84.240,88.260
7,82.540,89.500,82.070,90.650
8,82.900,88.700,82.390,89.350
9,82.760,88.770,82.170,90.110


In [336]:
# Multinomial
result = pd.DataFrame()
for i in range(10):
    X = raw.drop('spam', axis=1)
    y = raw.spam
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.6001, random_state=i)
    X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, train_size=0.5, random_state=i)
    
    lda_model = LDA().fit(X_train, y_train)
    lda_train_acc = get_acc(X_train, y_train, lda_model)
    lda_val_acc = get_acc(X_val, y_val, lda_model)
    
    nb_model = MultinomialNB().fit(X_train, y_train)
    nb_train_acc = get_acc(X_train, y_train, nb_model)
    nb_val_acc = get_acc(X_val, y_val, nb_model)
    result = result.append({'nb_train':nb_train_acc, 'lda_train':lda_train_acc, 'nb_val':nb_val_acc, 'lda_val':lda_val_acc}, ignore_index=True)
result_avg = result.mean()
result_avg.name = '평균'
result = result.append(result_avg)
result

,nb_train,lda_train,nb_val,lda_val
0,80.370,88.700,80.540,88.480
1,79.320,88.590,76.740,88.910
2,78.230,88.920,79.130,87.500
3,77.800,88.700,79.890,87.500
4,78.300,89.970,79.130,89.240
5,79.650,89.280,78.040,89.570
6,80.080,88.920,83.150,88.260
7,80.080,89.500,79.780,90.650
8,79.250,88.700,79.350,89.350
9,79.030,88.770,77.070,90.110


In [337]:
# Complement
result = pd.DataFrame()
for i in range(10):
    X = raw.drop('spam', axis=1)
    y = raw.spam
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.6001, random_state=i)
    X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, train_size=0.5, random_state=i)
    
    lda_model = LDA().fit(X_train, y_train)
    lda_train_acc = get_acc(X_train, y_train, lda_model)
    lda_val_acc = get_acc(X_val, y_val, lda_model)
    
    nb_model = ComplementNB().fit(X_train, y_train)
    nb_train_acc = get_acc(X_train, y_train, nb_model)
    nb_val_acc = get_acc(X_val, y_val, nb_model)
    result = result.append({'nb_train':nb_train_acc, 'lda_train':lda_train_acc, 'nb_val':nb_val_acc, 'lda_val':lda_val_acc}, ignore_index=True)
result_avg = result.mean()
result_avg.name = '평균'
result = result.append(result_avg)
result

,nb_train,lda_train,nb_val,lda_val
0,79.360,88.700,80.650,88.480
1,79.070,88.590,76.740,88.910
2,78.490,88.920,78.480,87.500
3,77.910,88.700,80.110,87.500
4,78.340,89.970,78.480,89.240
5,79.750,89.280,77.830,89.570
6,80.260,88.920,83.370,88.260
7,79.860,89.500,78.910,90.650
8,78.880,88.700,79.570,89.350
9,79.030,88.770,77.280,90.110


In [339]:
# Bernoulli
result = pd.DataFrame()
for i in range(10):
    X = raw.drop('spam', axis=1)
    y = raw.spam
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.6001, random_state=i)
    X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, train_size=0.5, random_state=i)
    
    lda_model = LDA().fit(X_train, y_train)
    lda_train_acc = get_acc(X_train, y_train, lda_model)
    lda_val_acc = get_acc(X_val, y_val, lda_model)
    lda_test_acc = get_acc(X_test, y_test, lda_model)
    
    nb_model = BernoulliNB().fit(X_train, y_train)
    nb_train_acc = get_acc(X_train, y_train, nb_model)
    nb_val_acc = get_acc(X_val, y_val, nb_model)
    nb_test_acc = get_acc(X_test, y_test, nb_model)
    result = result.append({'nb_train':nb_train_acc, 'lda_train':lda_train_acc, 'nb_val':nb_val_acc, 'lda_val':lda_val_acc, 'nb_test':nb_test_acc, 'lda_test': lda_test_acc}, ignore_index=True)
result_avg = result.mean()
result_avg.name = '평균'
result = result.append(result_avg)
result

,nb_train,lda_train,nb_val,lda_val,nb_test,lda_test
0,88.810,88.700,88.480,88.480,86.960,87.280
1,88.770,88.590,88.910,88.910,87.830,88.040
2,88.700,88.920,88.800,87.500,87.500,88.590
3,88.340,88.700,89.240,87.500,88.040,87.830
4,88.340,89.970,89.570,89.240,88.480,89.460
5,88.660,89.280,90.110,89.570,86.850,86.850
6,89.530,88.920,87.390,88.260,86.410,89.130
7,89.210,89.500,88.700,90.650,86.630,86.200
8,88.920,88.700,88.480,89.350,87.500,88.260
9,88.840,88.770,90.110,90.110,88.150,88.590
